## Uppgift 1 - EDA och data cleaning


## EDA

In [ ]:
import pandas as pd
from taxipred.utils.constants import TAXI_CSV_PATH

In [2]:
df = pd.read_csv(TAXI_CSV_PATH)

In [3]:
df

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180
...,...,...,...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,Weekend,4.0,Medium,Clear,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,Weekday,4.0,Medium,Clear,3.12,0.61,NaN,61.96,62.1295
997,7.70,Morning,Weekday,3.0,Low,Rain,2.08,1.78,NaN,54.18,33.1236
998,47.56,Morning,Weekday,1.0,Low,Clear,2.67,0.82,0.17,114.94,61.2090


In [4]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
count,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,951.000000
mean,27.070547,2.476842,3.502989,1.233316,0.292916,62.118116,56.874773
std,19.905300,1.102249,0.870162,0.429816,0.115592,32.154406,40.469791
min,1.230000,1.000000,2.010000,0.500000,0.100000,5.010000,6.126900
25%,12.632500,1.250000,2.730000,0.860000,0.190000,35.882500,33.742650
50%,25.830000,2.000000,3.520000,1.220000,0.290000,61.860000,50.074500
75%,38.405000,3.000000,4.260000,1.610000,0.390000,89.055000,69.099350
max,146.067047,4.000000,5.000000,2.000000,0.500000,119.840000,332.043689


I don't see any obvious outlier here, the max is a long trip sure, but with the rates for the trip makes sense and a longer trip is also reasonable so I don't see any reason to drop any of these.

In [5]:
df.columns

Index(['Trip_Distance_km', 'Time_of_Day', 'Day_of_Week', 'Passenger_Count',
       'Traffic_Conditions', 'Weather', 'Base_Fare', 'Per_Km_Rate',
       'Per_Minute_Rate', 'Trip_Duration_Minutes', 'Trip_Price'],
      dtype='object')

Columns seem pretty self explanatory, worth to note that Trip_Price = Base_Fare + (Trip_Distance_km * Per_Km_Rate) + (Trip_Duration_Minutes * Per_Minute_Rate).

In [ ]:
# Checking amount of NaN values.
df.isna().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

## Data Cleaning

From looking at the data, we can find some missing NaN values using math, in the columns Trip_Distance_km, Base_Fare, Per_Km_Rate, Per_Minute_Rate, Trip_Duration_Minutes and Trip_Price.

Formula to calculate missing Trip_Price:  
Trip_Price = Base_Fare + (Trip_Distance_km * Per_Km_Rate) + (Trip_Duration_Minutes * Per_Minute_Rate)

In [24]:
mask = (
    df["Trip_Price"].isna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Per_Km_Rate"].notna() &
    df["Trip_Duration_Minutes"].notna() &
    df["Per_Minute_Rate"].notna()
)

df.loc[mask, "Trip_Price"] = (
    df.loc[mask, "Base_Fare"]
    + df.loc[mask, "Trip_Distance_km"] * df.loc[mask, "Per_Km_Rate"]
    + df.loc[mask, "Trip_Duration_Minutes"] * df.loc[mask, "Per_Minute_Rate"]
)

In [25]:
#confirming
df.isna().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               17
dtype: int64

Formula to find missing Per_Km_Rate:  
Per_Km_Rate = (Trip_Price - Base_Fare - (Trip_Duration_Minutes × Per_Minute_Rate)) / Trip_Distance_km

In [26]:
mask = (
    df["Per_Km_Rate"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Trip_Duration_Minutes"].notna() &
    df["Per_Minute_Rate"].notna()
)

df.loc[mask, "Per_Km_Rate"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Duration_Minutes"] * df.loc[mask, "Per_Minute_Rate"])
    / df.loc[mask, "Trip_Distance_km"]
)

In [27]:
#confirming
df.isna().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              11
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               17
dtype: int64

Formula to find missing Per_Minute_Rate:  
Per_Minute_Rate = (Trip_Price - Base_Fare - (Trip_Distance_km × Per_Km_Rate)) / Trip_Duration_Minutes

In [28]:
mask = (
    df["Per_Minute_Rate"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Per_Km_Rate"].notna() &
    df["Trip_Duration_Minutes"].notna()
)

df.loc[mask, "Per_Minute_Rate"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Distance_km"] * df.loc[mask, "Per_Km_Rate"])
    / df.loc[mask, "Trip_Duration_Minutes"]
)

In [29]:
#confirming
df.isna().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              11
Per_Minute_Rate           5
Trip_Duration_Minutes    50
Trip_Price               17
dtype: int64

Formulas to calculate the remaining 3 columns that can be solved with math:  
Trip_Distance_km = (Trip_Price - Base_Fare - (Trip_Duration_Minutes × Per_Minute_Rate)) / Per_Km_Rate  
Trip_Duration_Minutes = (Trip_Price - Base_Fare - (Trip_Distance_km × Per_Km_Rate)) / Per_Minute_Rate  
Base_Fare = Trip_Price - (Trip_Distance_km × Per_Km_Rate) - (Trip_Duration_Minutes × Per_Minute_Rate)

In [30]:
# Calculating missing Trip_Distance_km
mask = (
    df["Trip_Distance_km"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Duration_Minutes"].notna() &
    df["Per_Minute_Rate"].notna() &
    df["Per_Km_Rate"].notna()
)

df.loc[mask, "Trip_Distance_km"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Duration_Minutes"] * df.loc[mask, "Per_Minute_Rate"])
    / df.loc[mask, "Per_Km_Rate"]
)

In [31]:
# Calculating missing Trip_Duration_Minutes
mask = (
    df["Trip_Duration_Minutes"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Per_Km_Rate"].notna() &
    df["Per_Minute_Rate"].notna()
)

df.loc[mask, "Trip_Duration_Minutes"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Distance_km"] * df.loc[mask, "Per_Km_Rate"])
    / df.loc[mask, "Per_Minute_Rate"]
)

In [32]:
# Calculating missing Base_Fare
mask = (
    df["Base_Fare"].isna() &
    df["Trip_Price"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Trip_Duration_Minutes"].notna() &
    df["Per_Km_Rate"].notna() &
    df["Per_Minute_Rate"].notna()
)

df.loc[mask, "Base_Fare"] = (
    df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Trip_Distance_km"] * df.loc[mask, "Per_Km_Rate"]
    - df.loc[mask, "Trip_Duration_Minutes"] * df.loc[mask, "Per_Minute_Rate"]
)

In [33]:
#confirming
df.isna().sum()

Trip_Distance_km          6
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                12
Per_Km_Rate              11
Per_Minute_Rate           5
Trip_Duration_Minutes    15
Trip_Price               17
dtype: int64

Since we have solved all missing NaN values we can using math, I think it now makes sense to take the average of Base_Fare to replace the 12 NaN values in that column, then we can run the calculations again and get results that are "close enough".

In [34]:
df["Base_Fare"].describe()

count    988.000000
mean       3.498806
std        0.873047
min        2.010000
25%        2.730000
50%        3.510000
75%        4.260000
max        5.000000
Name: Base_Fare, dtype: float64

In [35]:
# Replacing Base_Fare NaN values with 3.50 (rounded up mean of the column).
df["Base_Fare"] = df["Base_Fare"].fillna(3.50)

In [36]:
#confirming
df.isna().sum()

Trip_Distance_km          6
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                 0
Per_Km_Rate              11
Per_Minute_Rate           5
Trip_Duration_Minutes    15
Trip_Price               17
dtype: int64

Now that we have 0 NaN values in Base_Fare, we can run some calculations again that failed due to there being no Base_Fare value in that row before. Since the difference between min and max of Base_Fare is so small I think its justifiable to do it this way, as our calculations will have a 3 dollar margin of error.

In [37]:
# Trip_Price...
mask = (
    df["Trip_Price"].isna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Per_Km_Rate"].notna() &
    df["Trip_Duration_Minutes"].notna() &
    df["Per_Minute_Rate"].notna()
)

df.loc[mask, "Trip_Price"] = (
    df.loc[mask, "Base_Fare"]
    + df.loc[mask, "Trip_Distance_km"] * df.loc[mask, "Per_Km_Rate"]
    + df.loc[mask, "Trip_Duration_Minutes"] * df.loc[mask, "Per_Minute_Rate"]
)

In [38]:
# Per_Km_Rate...
mask = (
    df["Per_Km_Rate"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Trip_Duration_Minutes"].notna() &
    df["Per_Minute_Rate"].notna()
)

df.loc[mask, "Per_Km_Rate"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Duration_Minutes"] * df.loc[mask, "Per_Minute_Rate"])
    / df.loc[mask, "Trip_Distance_km"]
)

In [39]:
# Per_Minute_Rate...
mask = (
    df["Per_Minute_Rate"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Per_Km_Rate"].notna() &
    df["Trip_Duration_Minutes"].notna()
)

df.loc[mask, "Per_Minute_Rate"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Distance_km"] * df.loc[mask, "Per_Km_Rate"])
    / df.loc[mask, "Trip_Duration_Minutes"]
)

In [40]:
# Trip_Distance_km...
mask = (
    df["Trip_Distance_km"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Duration_Minutes"].notna() &
    df["Per_Minute_Rate"].notna() &
    df["Per_Km_Rate"].notna()
)

df.loc[mask, "Trip_Distance_km"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Duration_Minutes"] * df.loc[mask, "Per_Minute_Rate"])
    / df.loc[mask, "Per_Km_Rate"]
)

In [41]:
# Trip_Duration_Minutes...
mask = (
    df["Trip_Duration_Minutes"].isna() &
    df["Trip_Price"].notna() &
    df["Base_Fare"].notna() &
    df["Trip_Distance_km"].notna() &
    df["Per_Km_Rate"].notna() &
    df["Per_Minute_Rate"].notna()
)

df.loc[mask, "Trip_Duration_Minutes"] = (
    (df.loc[mask, "Trip_Price"]
    - df.loc[mask, "Base_Fare"]
    - df.loc[mask, "Trip_Distance_km"] * df.loc[mask, "Per_Km_Rate"])
    / df.loc[mask, "Per_Minute_Rate"]
)

In [42]:
#confirming
df.isna().sum()

Trip_Distance_km          5
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                 0
Per_Km_Rate              10
Per_Minute_Rate           5
Trip_Duration_Minutes    11
Trip_Price               11
dtype: int64

We could use the same method of taking the average of the columns that still have missing values, but for an example if we are 50 cent of Per_Minute_Rate on an 80 min taxi ride then our Trip_Price will be 40 dollars of, which feels like a lot compared to the margin of error only using this method for Base_Fare. Therefore I will just drop the remaining NaN values in the columns we have been running the calculations for.  
As for the remaining columns with NaN values that cannot be filled using math, I will just drop them too, so moving forward I will just drop the remaining rows with any NaN value in them and I'm satisfied with my data cleaning.

In [ ]:
# Dropping all remaining NaN values.
df = df.dropna()

In [ ]:
#confirming
df.isna().sum()

Trip_Distance_km         0
Time_of_Day              0
Day_of_Week              0
Passenger_Count          0
Traffic_Conditions       0
Weather                  0
Base_Fare                0
Per_Km_Rate              0
Per_Minute_Rate          0
Trip_Duration_Minutes    0
Trip_Price               0
dtype: int64

In [47]:
# Exporting to csv to use for Uppgift 2 - ML model
df.to_csv("../data/cleaned_taxi_data.csv", index=False)